### Read Multiple Result Map

In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
from absbox import API,EnginePath
localAPI = API(EnginePath.DEV,check=False)


from absbox.examples import test01,test02

Connecting engine server -> https://absbox.org/api/dev

✅Connected, local lib:0.28.7, server:0.28.18

#### What is MultiResult ?

`Multi Result` is just a map result return either by `runByScenarios()` or `runStructs()`

In [4]:
rs1 = localAPI.runStructs({"deal1":test01,"deal2":test02}
                         ,read=True)

In [13]:
p1 = ("Pool",("Mortgage",None,{"CPR":0.01},None,None)
                                ,None
                                ,None)

p2 = ("Pool",("Mortgage",None,{"CPR":0.02},None,None)
                                ,None
                                ,None)


rs2 = localAPI.runByScenarios(test01
                             ,{"CPR=1%":p1,"CPR=2%":p2}
                             ,runAssump = [
                                 ("pricing"
                                  ,{"date":"2021-08-22"
                                    ,"curve":[["2021-01-01",0.025]
                                             ,["2024-08-01",0.025]]})
                             ]
                             ,read=True)

User need to import candy functions and `lens` to build paths which points to the data field of interest

In [6]:
from absbox import readFlowsByScenarios,readMultiFlowsByScenarios,readFieldsByScenarios

from lenses import lens

#### readFlowsByScenarios()

The read single cashflow from multiple scenarios and build a dataframe

In [7]:
readFlowsByScenarios(rs1, lens['bonds']['A1']['balance']).head()

,deal1:balance,deal2:balance
date,,
2021-06-20,543.90,340.83
2021-07-20,429.17,224.94
2021-08-20,313.86,108.41
2021-09-20,197.87,0.00
2021-10-20,81.15,NaN


In [8]:
readFlowsByScenarios(rs1, lens['bonds']['A1']['balance'], fullName=False).head()

,balance,balance
date,,
2021-06-20,543.90,340.83
2021-07-20,429.17,224.94
2021-08-20,313.86,108.41
2021-09-20,197.87,0.00
2021-10-20,81.15,NaN


In [9]:
readFlowsByScenarios(rs2, lens['pool']['flow']['Prepayment']).head()

,CPR=1%:Prepayment,CPR=2%:Prepayment
Date,,
2021-04-01,3.47,6.98
2021-05-01,1.64,3.29
2021-06-01,1.60,3.22
2021-07-01,1.46,2.93
2021-08-01,1.42,2.85


#### readMultiFlowsByScenarios

User can have option to view `multiple columns` from a multi result and merge them into a multi-index dataframe

In [10]:
readMultiFlowsByScenarios(rs2
                          , (lens['pool']['flow'],["Balance",'Prepayment'])).head()

Scenario     CPR=1%              CPR=2%           
Field       Balance Prepayment  Balance Prepayment
Date                                              
2021-04-01  1989.69       3.47  1986.45       6.98
2021-05-01  1883.73       1.64  1879.09       3.29
2021-06-01  1777.20       1.60  1771.29       3.22
2021-07-01  1670.20       1.46  1663.26       2.93
2021-08-01  1562.63       1.42  1554.79       2.85

#### readFieldsByScenarios

| since `0.28.15`

`readFieldsByScenarios` is more flexible candy function to view the fields from multiple results

Here is the dataframe to be view

In [ ]:
rs2['CPR=1%']['pricing']['summary']

view by `Index`

In [92]:
readFieldsByScenarios(rs2, lens['pricing']['summary'], "A1")

,CPR=1%,CPR=2%
pricing,308.40,300.610
face,30.84,30.061
WAL,0.15,0.140
duration,0.14,0.140
convexity,0.16,0.160
accure interest,0.11,0.110


view by `Column`

In [93]:
readFieldsByScenarios(rs2, lens['pricing']['summary'], "pricing", flip=True)

,CPR=1%,CPR=2%
A1,308.40,300.61
B,1308.17,1307.59


view by `Column` ,which is a flow

In [94]:
readFieldsByScenarios(rs2, lens['pool']['flow'], 'Prepayment', flip=True)

,CPR=1%,CPR=2%
Date,,
2021-04-01,3.47,6.98
2021-05-01,1.64,3.29
2021-06-01,1.6,3.22
2021-07-01,1.46,2.93
2021-08-01,1.42,2.85
2021-09-01,1.33,2.66
2021-10-01,1.2,2.39
2021-11-01,1.14,2.29
2021-12-01,1.02,2.03


view by `Index` which is a flow

In [95]:
readFieldsByScenarios(rs2, lens['pool']['flow'], '2021-10-01')

,CPR=1%,CPR=2%
Balance,1345.93,1336.93
Principal,107.4,106.67
Interest,9.68,9.62
Prepayment,1.2,2.39
Default,0,0
Recovery,0,0
Loss,0,0
WAC,0.08,0.08
BorrowerNum,None,None
PrepayPenalty,None,None
